In [ ]:
!pip install datasets transformers -U

from datasets import load_dataset

dataset = load_dataset("imdb")
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

train_dataset = dataset["train"].map(tokenize_function, batched=True)
test_dataset = dataset["test"].map(tokenize_function, batched=True) 
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModel

en_to_de_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
en_to_de_model = AutoModel.from_pretrained("Helsinki-NLP/opus-mt-en-de")

de_to_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
de_to_en_model = AutoModel.from_pretrained("Helsinki-NLP/opus-mt-de-en")

Some weights of the model checkpoint at Helsinki-NLP/opus-mt-en-de were not used when initializing MarianModel: ['final_logits_bias']
- This IS expected if you are initializing MarianModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MarianModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at Helsinki-NLP/opus-mt-de-en were not used when initializing MarianModel: ['final_logits_bias']
- This IS expected if you are initializing MarianModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializin

In [ ]:
import torch
import torch.nn.functional as F

def unsupervised_loss(logits_orig, logits_aug, temperature=1.0, device="cuda"):
    prob_orig = F.softmax(logits_orig / temperature, dim=1)
    prob_aug = F.softmax(logits_aug / temperature, dim=1)
    
    loss = F.kl_div(torch.log(prob_aug), prob_orig, reduction="batchmean")
    return loss


In [ ]:
import numpy as np
from transformers import AdamW
from torch.utils.data import DataLoader

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

device = "cuda" if torch.cuda.is_available() else "cpu"
import torch
import torch.nn.functional as F
from transformers import MarianTokenizer, MarianMTModel, AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader

# Define the back_translate function
def back_translate(texts, source_tokenizer, source_model, target_tokenizer, target_model):
    source_texts = source_tokenizer.prepare_seq2seq_batch(src_texts=texts, return_tensors="pt")
    source_texts.to(device)

    with torch.no_grad():
        translated_texts = source_model.generate(**source_texts)

    translated_texts = [source_tokenizer.decode(t, skip_special_tokens=True) for t in translated_texts]

    target_texts = target_tokenizer.prepare_seq2seq_batch(src_texts=translated_texts, return_tensors="pt")
    target_texts.to(device)

    with torch.no_grad():
        back_translated_texts = target_model.generate(**target_texts)

    return [target_tokenizer.decode(t, skip_special_tokens=True) for t in back_translated_texts]

# Load the translation models
en_to_de_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
en_to_de_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-de").to(device)

de_to_en_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
de_to_en_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-de-en").to(device)

model = model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 3
labeled_batch_size = 16
unlabeled_batch_size = 32
alpha = 0.5
temperature = 1.0

labeled_data_loader = DataLoader(train_dataset, batch_size=labeled_batch_size, shuffle=True)
unlabeled_data_loader = DataLoader(train_dataset, batch_size=unlabeled_batch_size, shuffle=True)

for epoch in range(num_epochs):
    model.train()
    for batch_labeled, batch_unlabeled in zip(labeled_data_loader, unlabeled_data_loader):
        optimizer.zero_grad()
        
        # Supervised loss
        input_ids = batch_labeled["input_ids"].to(device)
        attention_mask = batch_labeled["attention_mask"].to(device)
        labels = batch_labeled["label"].to(device)
        logits = model(input_ids, attention_mask=attention_mask).logits
        loss_supervised = F.cross_entropy(logits, labels)
        
        # Unsupervised loss
        input_ids = batch_unlabeled["input_ids"].to(device)
        attention_mask = batch_unlabeled["attention_mask"].to(device)
        logits_orig = model(input_ids, attention_mask=attention_mask).logits.detach()
        
        texts = [tokenizer.decode(input_id) for input_id in input_ids.cpu().numpy()]
        texts_back_translated = back_translate(texts, en_to_de_tokenizer, en_to_de_model, de_to_en_tokenizer, de_to_en_model)
        input_ids_back_translated = tokenizer(texts_back_translated, return_tensors="pt", padding=True, truncation=True)["input_ids"].to(device)
        attention_mask_back_translated = (input_ids_back_translated != tokenizer.pad_token_id).to(device)
        
        logits_aug = model(input_ids_back_translated, attention_mask=attention_mask_back_translated).logits
        loss_unsupervised = unsupervised_loss(logits_orig, logits_aug, temperature=temperature)
        
        # Total loss
        loss = loss_supervised + alpha * loss_unsupervised
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs} finished.")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3712: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the

Epoch 1/3 finished.
Epoch 2/3 finished.
Epoch 3/3 finished.


In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(model, tokenizer, test_dataset, device="cuda"):
    model.eval()
    test_data_loader = DataLoader(test_dataset, batch_size=32)
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in test_data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            logits = model(input_ids, attention_mask=attention_mask).logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            true_labels.extend(labels.cpu().numpy())
            predictions.extend(preds)

    accuracy = accuracy_score(true_labels, predictions)
    return accuracy

test_accuracy = evaluate(model, tokenizer, test_dataset)
print(f"Test accuracy: {test_accuracy:.3f}")


Test accuracy: 0.928
